In [1]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, FloatType
)
from pyspark.sql.functions import col, when
from pyspark.storagelevel import StorageLevel
from typing import Dict


class SparkCSVToParquet:
    def __init__(self, app_name: str):
        load_dotenv()
        self.spark = self._create_spark_session(app_name)
        print(f"[INFO] Spark session started: {app_name}")

    def _create_spark_session(self, app_name: str) -> SparkSession:
        aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
        aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

        return SparkSession.builder \
            .appName(app_name) \
            .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
            .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
            .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
            .getOrCreate()

    def read_csv(self, path: str, schema: StructType, delimiter: str = ";") -> DataFrame:
        print(f"[INFO] Reading CSV: {path}")
        return self.spark.read \
            .option("header", True) \
            .option("delimiter", delimiter) \
            .schema(schema) \
            .csv(path)

    def transform_column_to_date(self, df: DataFrame, column_name: str) -> DataFrame:
        print(f"[INFO] Casting column '{column_name}' to DateType")
        return df.withColumn(column_name, col(column_name).cast("date"))

    def cache_and_persist(self, df: DataFrame) -> DataFrame:
        return df.cache().persist(StorageLevel.MEMORY_AND_DISK)

    def save_as_parquet(self, df: DataFrame, path: str, mode: str = "overwrite"):
        print(f"[INFO] Saving DataFrame to Parquet: {path}")
        df.write.mode(mode).parquet(path)

    def stop(self):
        print("[INFO] Stopping Spark session")
        self.spark.stop()


if __name__ == "__main__":
    processor = SparkCSVToParquet("CSV to Parquet Pipeline")

    schemas: Dict[str, StructType] = {
        "card": StructType([
            StructField("card_id", IntegerType(), True),
            StructField("disp_id", IntegerType(), True),
            StructField("type", StringType(), True),
            StructField("issued", StringType(), True),
        ]),
        "client": StructType([
            StructField("client_id", IntegerType(), True),
            StructField("gender", StringType(), True),
            StructField("birth_date", StringType(), True),
            StructField("district_id", IntegerType(), True),
        ]),
        "disp": StructType([
            StructField("disp_id", IntegerType(), True),
            StructField("client_id", IntegerType(), True),
            StructField("account_id", IntegerType(), True),
            StructField("type", StringType(), True),
        ]),
        "account": StructType([
            StructField("account_id", IntegerType(), True),
            StructField("district_id", IntegerType(), True),
            StructField("frequency", StringType(), True),
            StructField("date", StringType(), True),
        ]),
        "district": StructType([
            StructField("district_id", IntegerType(), True),
            StructField("district_name", StringType(), True),
            StructField("region", StringType(), True),
            StructField("population", IntegerType(), True),
            StructField("A2", IntegerType(), True),
            StructField("A3", IntegerType(), True),
            StructField("A4", IntegerType(), True),
            StructField("A5", IntegerType(), True),
            StructField("A6", IntegerType(), True),
            StructField("A7", IntegerType(), True),
            StructField("A8", IntegerType(), True),
            StructField("A9", IntegerType(), True),
            StructField("A10", FloatType(), True),
            StructField("A11", FloatType(), True),
            StructField("A12", IntegerType(), True),
            StructField("A13", IntegerType(), True),
            StructField("A14", IntegerType(), True),
            StructField("A15", IntegerType(), True),
            StructField("A16", IntegerType(), True),
        ]),
        "loan": StructType([
            StructField("loan_id", IntegerType(), True),
            StructField("account_id", IntegerType(), True),
            StructField("date", StringType(), True),
            StructField("amount", IntegerType(), True),
            StructField("duration", IntegerType(), True),
            StructField("payments", FloatType(), True),
            StructField("status", StringType(), True)
        ]),
        "order": StructType([
            StructField("order_id", IntegerType(), True),
            StructField("account_id", IntegerType(), True),
            StructField("bank_to", StringType(), True),
            StructField("account_to", IntegerType(), True),
            StructField("amount", FloatType(), True),
            StructField("k_symbol", StringType(), True)
        ]),
        # Add schema for trans
        "trans": StructType([
            StructField("trans_id", IntegerType(), True),
            StructField("account_id", IntegerType(), True),
            StructField("date", StringType(), True),
            StructField("type", StringType(), True),
            StructField("operation", StringType(), True),
            StructField("amount", FloatType(), True),
            StructField("balance", FloatType(), True),
            StructField("k_symbol", StringType(), True),
            StructField("branch", StringType(), True),
            StructField("bank", StringType(), True),
            StructField("account", IntegerType(), True)
        ])
    }

    sources = {
        "card": {
            "path": "s3a://nmourmx-scigility/Bronze/card/card.csv",
            "date_col": "issued",
            "output": "s3a://nmourmx-scigility/Silver/card_parquet/",
        },
        "client": {
            "path": "s3a://nmourmx-scigility/Bronze/client/client.csv",
            "date_col": "birth_date",
            "output": "s3a://nmourmx-scigility/Silver/client_parquet/",
        },
        "disp": {
            "path": "s3a://nmourmx-scigility/Bronze/disp/disp.csv",
            "date_col": None,
            "output": "s3a://nmourmx-scigility/Silver/disp_parquet/",
        },
        "account": {
            "path": "s3a://nmourmx-scigility/Bronze/account/account.csv",
            "date_col": "date",
            "output": "s3a://nmourmx-scigility/Silver/account_parquet/",
        },
        "district": {
            "path": "s3a://nmourmx-scigility/Bronze/district/district.csv",
            "date_col": None,
            "output": "s3a://nmourmx-scigility/Silver/district_parquet/",
        },
        "loan": {
            "path": "s3a://nmourmx-scigility/Bronze/loan/loan.csv",
            "date_col": "date",
            "output": "s3a://nmourmx-scigility/Silver/loan_parquet/",
        },
        "order": {
            "path": "s3a://nmourmx-scigility/Bronze/order/order.csv",
            "date_col": None,
            "output": "s3a://nmourmx-scigility/Silver/order_parquet/",
        },
        # Add trans source
        "trans": {
            "path": "s3a://nmourmx-scigility/Bronze/trans/trans.csv",
            "date_col": "date",
            "output": "s3a://nmourmx-scigility/Silver/trans_fixed_parquet/",
        }
    }

    for name, config in sources.items():
        print(f"\n========== Processing {name.upper()} ==========")
        df = processor.read_csv(config["path"], schemas[name])
        if config["date_col"]:
            df = processor.transform_column_to_date(df, config["date_col"])

        # Special fix for trans type typo
        if name == "trans":
            from pyspark.sql.functions import when
            print("[INFO] Fixing typo in 'type' column for trans dataset")
            df = df.withColumn("type", when(col("type") == "PRJIEM", "PRIJEM").otherwise(col("type")))
            # Select distinct values from the 'type' column
            distinct_types = df.select("type").distinct()      
            # Show the distinct values
            distinct_types.show()

        df = processor.cache_and_persist(df)
        df.show(10)
        processor.save_as_parquet(df, config["output"])

    processor.stop()


25/07/29 11:36:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[INFO] Spark session started: CSV to Parquet Pipeline

========== Processing CARD ==========
[INFO] Reading CSV: s3a://nmourmx-scigility/Bronze/card/card.csv


25/07/29 11:36:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


[INFO] Casting column 'issued' to DateType


25/07/29 11:37:02 WARN CacheManager: Asked to cache already cached data.


...


========== Processing TRANS ==========
[INFO] Reading CSV: s3a://nmourmx-scigility/Bronze/trans/trans.csv
[INFO] Casting column 'date' to DateType
[INFO] Fixing typo in 'type' column for trans dataset


25/07/29 11:37:31 WARN CacheManager: Asked to cache already cached data.        


+------+
|  type|
+------+
| VYDAJ|
|PRIJEM|
| VYBER|
+------+



+--------+----------+----------+------+--------------+-------+-------+--------+------+----+--------+
|trans_id|account_id|      date|  type|     operation| amount|balance|k_symbol|branch|bank| account|
+--------+----------+----------+------+--------------+-------+-------+--------+------+----+--------+
|  967842|      1532|1998-05-31|PRIJEM|          NULL|  253.0|62661.0|    UROK|    AR|NULL|       0|
|  271012|      1499|1998-01-09|PRIJEM|         VKLAD| 5500.0|49790.0|    NULL|    AR|NULL|       0|
|  971490|      1645|1996-11-30|PRIJEM|          NULL|   43.0|19460.0|    UROK|    AR|NULL|       0|
|  605699|      3366|1996-10-28| VYDAJ|         VYBER| 9300.0|38433.0|    NULL|    AR|NULL|       0|
|  443107|      2471|1998-07-30| VYDAJ|         VYBER| 7800.0|56237.0|    NULL|    AR|NULL|       0|
|  627433|      3503|1995-06-07| VYDAJ|         VYBER|28640.0|41949.0|    NULL|    AR|NULL|       0|
|  390740|      2162|1997-12-03|PRIJEM|         VKLAD|11111.0|38640.0|    NULL|    AR|NULL|

[INFO] Stopping Spark session
